In [1]:
import sys
sys.path.append('/Users/dsmith/Documents/code/alma_batch')

In [2]:
from alma_batch import AlmaBatch
import pandas as pd
from pathlib import Path

Load a set of items to be scanned in from Alma.

In [3]:
PATH_TO_CONFIG = Path('./')

In [4]:
set_id = '14218981390004107'
set_config_file = PATH_TO_CONFIG / 'sets_config.yml'

In [5]:
set_batch = AlmaBatch(set_config_file)

In [6]:
set_batch.data = [{'set_id': set_id}]
set_batch.validate_data()

In [7]:
# Fetch contents of the set
await set_batch.amake_requests()

In [8]:
# Flatten list
members = [member for result in set_batch.results for member in result['result']['member']]

In [9]:
len(members)

86

Scan the items using the Bibs API

In [10]:
items_config_file = PATH_TO_CONFIG / 'scan_items_config.yml'

In [11]:
# Extract links and add additional params 
scan_in_params = {'op': 'scan',
                 'library': 'gelman',
                 'circ_desk': 'SCRC_DESK'}
items_to_scan = [dict({'link': item['link']}, **scan_in_params)
                for item in members]

In [12]:
scan_batch = AlmaBatch(items_config_file)
scan_batch.data = items_to_scan
scan_batch.validate_data()

In [13]:
await scan_batch.amake_requests()

In [14]:
scan_batch.errors

[]

Load a CSV.

In [26]:
migration_errors = pd.read_csv('items_technical_migration.csv')

Make sure the relevant column names match the parameters of the API endpoint.

In [4]:
migration_errors.columns = [column.lower().replace(' ', '_') for column in migration_errors.columns]

In [10]:
migration_errors = migration_errors.rename(columns={'item_id': 'item_pid'})

This is the path to your config file.

In [12]:
CONFIG_PATH = './test_config.yml'

Initialize the AlmaBatch object.

In [13]:
ab = AlmaBatch(CONFIG_PATH)

Pass in the data.

In [21]:
ab.data = migration_errors.copy()

Optional: checks that all parameters in the API URL are present in the columns of the `data` attribute.

In [22]:
ab.validate_data()

From a notebook, use the async function directly: `AlmaBatch.amake_requests`. From a standalone script, simply do
`ab.make_requests(batch_size=500, serialize_return=True)`, which calls a wrapper function that kicks off the event loop. (In a Jupyter environment, we use the already running event loop.)

Set `serialize_return` to `False` if you don't want to keep the API output. 

Set `batch_size` to `0` if you want to run all rows at once.

In [23]:
await ab.amake_requests(batch_size=500, serialize_return=True)

Running batch 1...
Saving API output for batch 0
All requests completed.


Errors can be inspected via the `AlmaBatch.errors` attribute.

In [25]:
ab.errors[0]

{'item_pid': 23406647760004107,
 'barcode': nan,
 'base_status': 'Item not in place',
 'process_type': 'Technical - Migration',
 'internal_note_1': nan,
 'item_call_number': nan,
 'holding_id': 22406647770004107,
 'suppressed_holdings': 0,
 'permanent_call_number': 'ND813.Z96 G79',
 'location_name': 'Stacks',
 'temporary_location_name': 'None',
 'title': '[Homenaje a Zurbarán].',
 'mms_id': 9927473753604107,
 'suppressed_bibs': 'No',
 'material_type': 'Book',
 'originating_system_id': '2747375-wrlcdb',
 'idx': 34,
 'error': alma_batch_utils.APIException('{"errorsExist":true,"errorList":{"error":[{"errorCode":"401683","errorMessage":"No Item found for mmsId 9927473753604107, holdings id 22406647770004107 and item id 23406647760004107.","trackingId":"E01-0610153857-7NPIS-AWAE2090425221"}]},"result":null}')}